импорт библиотек

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go 

In [2]:
# Чтение данных из Excel
df = pd.read_excel("data.xlsx")

# Удаление пробелов в начале и в конце строк для столбцов "Источник" и "Магистраль"
df['Источник'] = df['Источник'].str.strip()
df['Магистраль'] = df['Магистраль'].str.strip()

# Создание нового столбца с комбинацией "Источник+Магистраль"
df['Источник+Магистраль'] = df['Источник'] + '+' + df['Магистраль']

# Преобразование столбца "Дата" в формат datetime
df['Дата'] = pd.to_datetime(df['Дата'], errors='coerce')

# Извлечение уникальных дат (только даты)
unique_dates = sorted(df['Дата'].dropna().dt.date.unique())

# Извлечение уникальных комбинаций "Источник+Магистраль"
unique_combinations = df['Источник+Магистраль'].dropna().unique()

In [3]:
# Создание пустых DataFrame для:
# 1. Номеров записей
# 2. Температуры наружного воздуха (Тнв)
# 3. Фактической температуры наружного воздуха (Тнвф)
work_days_df = pd.DataFrame(index=unique_dates, columns=unique_combinations)
outer_temperature_df = pd.DataFrame(index=unique_dates, columns=unique_combinations)
actual_temperature_df = pd.DataFrame(index=unique_dates, columns=unique_combinations)

water_direct_pred_df  = pd.DataFrame(index=unique_dates, columns=unique_combinations)  # прямая по графику (прогноз) – 5-ый столбец (индекс 4)
water_reverse_pred_df = pd.DataFrame(index=unique_dates, columns=unique_combinations)  # обратная по графику (прогноз) – 6-ой столбец (индекс 5)
water_direct_act_df   = pd.DataFrame(index=unique_dates, columns=unique_combinations)  # прямая по факту – 7-ой столбец (индекс 6)
water_reverse_act_df  = pd.DataFrame(index=unique_dates, columns=unique_combinations)  # обратная по факту – 8-ой столбец (индекс 7)

# Заполнение всех таблиц
for date in unique_dates:
    filtered_by_date = df[df['Дата'].dt.date == date]
    for combination in unique_combinations:
        filtered_by_combination = filtered_by_date[filtered_by_date['Источник+Магистраль'] == combination]
        if not filtered_by_combination.empty:
            # Номера записей и температуры наружного воздуха
            work_days_df.at[date, combination] = filtered_by_combination.iloc[0]['№№']
            outer_temperature_df.at[date, combination] = filtered_by_combination.iloc[0]['Тнв']
            actual_temperature_df.at[date, combination] = filtered_by_combination.iloc[0, 13]  # фактическая Тнв (14-ый столбец, индекс 13)
            
            # Температура теплоносителя (воды)
            water_direct_pred_df.at[date, combination]  = filtered_by_combination.iloc[0, 4]  # 5-ый столбец: прямая (прогноз)
            water_reverse_pred_df.at[date, combination] = filtered_by_combination.iloc[0, 5]  # 6-ой столбец: обратная (прогноз)
            water_direct_act_df.at[date, combination]   = filtered_by_combination.iloc[0, 6]  # 7-ой столбец: прямая (факт)
            water_reverse_act_df.at[date, combination]  = filtered_by_combination.iloc[0, 7]  # 8-ой столбец: обратная (факт)

In [4]:
with pd.ExcelWriter("output.xlsx", engine='xlsxwriter') as writer:
    # Листы для наружного воздуха
    work_days_df.to_excel(writer, sheet_name="Номера записей")
    outer_temperature_df.to_excel(writer, sheet_name="Тнв")
    actual_temperature_df.to_excel(writer, sheet_name="Тнвф")
    
    # Листы для теплоносителя (воды)
    water_direct_pred_df.to_excel(writer, sheet_name="Вода прямая (прогноз)")
    water_reverse_pred_df.to_excel(writer, sheet_name="Вода обратная (прогноз)")
    water_direct_act_df.to_excel(writer, sheet_name="Вода прямая (факт)")
    water_reverse_act_df.to_excel(writer, sheet_name="Вода обратная (факт)")
    
    # Получаем объекты книги и листов для форматирования
    workbook = writer.book
    worksheet_tnv   = writer.sheets["Тнв"]
    worksheet_tnvf  = writer.sheets["Тнвф"]
    worksheet_water_direct_pred  = writer.sheets["Вода прямая (прогноз)"]
    worksheet_water_reverse_pred = writer.sheets["Вода обратная (прогноз)"]
    worksheet_water_direct_act   = writer.sheets["Вода прямая (факт)"]
    worksheet_water_reverse_act  = writer.sheets["Вода обратная (факт)"]
    
    # Формат для выделения выбросов (красная заливка)
    red_format = workbook.add_format({'bg_color': '#FFC7CE'})

In [5]:
# Функция для выделения выбросов по строкам (метод 2σ)
def highlight_outliers(worksheet, df_sheet, start_row=1, start_col=1):
    for i, date in enumerate(df_sheet.index):
        row_series = df_sheet.loc[date]
        row_values = pd.to_numeric(row_series, errors='coerce')
        valid = row_values.dropna()
        if len(valid) < 2:
            continue
        row_mean = valid.mean()
        row_std  = valid.std()
        if row_std == 0:
            continue
        for j, col in enumerate(df_sheet.columns):
            cell_value = row_series[col]
            try:
                value = float(cell_value)
            except (ValueError, TypeError):
                continue
            if abs(value - row_mean) > 2 * row_std:
                worksheet.write(i + start_row, j + start_col, value, red_format)
    


In [6]:
highlight_outliers(worksheet_tnv, outer_temperature_df)
highlight_outliers(worksheet_tnvf, actual_temperature_df)

# Выделение выбросов для теплоносителя (воды)
highlight_outliers(worksheet_water_direct_pred, water_direct_pred_df)
highlight_outliers(worksheet_water_reverse_pred, water_reverse_pred_df)
highlight_outliers(worksheet_water_direct_act, water_direct_act_df)
highlight_outliers(worksheet_water_reverse_act, water_reverse_act_df)

In [ ]:
# Приводим данные к числовому типу для наружного воздуха
outer_numeric  = outer_temperature_df.apply(pd.to_numeric, errors='coerce')
actual_numeric = actual_temperature_df.apply(pd.to_numeric, errors='coerce')

# 1. Линейный график: сравнение дневных средних температур наружного воздуха
daily_predicted = outer_numeric.mean(axis=1)
daily_actual    = actual_numeric.mean(axis=1)
comparison_df = pd.DataFrame({
    'Дата': daily_predicted.index,
    'Тнв (метеорологи)': daily_predicted.values,
    'Тнвф (фактическая)': daily_actual.values
})
comparison_df['Дата'] = pd.to_datetime(comparison_df['Дата'])

fig_line = px.line(
    comparison_df,
    x='Дата',
    y=['Тнв (метеорологи)', 'Тнвф (фактическая)'],
    title="Сравнение дневной средней температуры наружного воздуха: метеорологи vs фактическая",
    labels={'value': 'Температура (°C)', 'variable': 'Тип данных'}
)
fig_line.write_html("graphs/line_comparison_outer_temp.html")

In [ ]:
# 2. Scatter plot для наружного воздуха
stacked_predicted = outer_numeric.stack().reset_index(name='Тнв')
stacked_actual    = actual_numeric.stack().reset_index(name='Тнвф')
merged = pd.merge(stacked_predicted, stacked_actual, on=['level_0', 'level_1'])
merged = merged.rename(columns={'level_0': 'Дата', 'level_1': 'Комбинация'})
merged['Дата'] = pd.to_datetime(merged['Дата'])

fig_scatter = px.scatter(
    merged,
    x='Тнв',
    y='Тнвф',
    color='Комбинация',
    hover_data=['Дата', 'Комбинация'],
    title="Сравнение по точкам: метеорологи (Тнв) vs фактическая (Тнвф)",
    labels={'Тнв': 'Тнв (метеорологи)', 'Тнвф': 'Тнвф (фактическая)'}
)
min_val = min(merged['Тнв'].min(), merged['Тнвф'].min())
max_val = max(merged['Тнв'].max(), merged['Тнвф'].max())
fig_scatter.add_trace(
    go.Scatter(
        x=[min_val, max_val],
        y=[min_val, max_val],
        mode='lines',
        line=dict(dash='dash', color='red'),
        name='y=x'
    )
)
fig_scatter.write_html("graphs/scatter_comparison_outer_temp.html")

In [11]:
# Приводим данные к числовому типу для воды (прямая и обратная)
water_direct_pred_numeric  = water_direct_pred_df.apply(pd.to_numeric, errors='coerce')
water_direct_act_numeric   = water_direct_act_df.apply(pd.to_numeric, errors='coerce')
water_reverse_pred_numeric = water_reverse_pred_df.apply(pd.to_numeric, errors='coerce')
water_reverse_act_numeric  = water_reverse_act_df.apply(pd.to_numeric, errors='coerce')

## --- Графики для прямой линии воды (прогноз vs факт) ---
## Линейный график

In [ ]:
daily_water_direct_pred = water_direct_pred_numeric.mean(axis=1)
daily_water_direct_act  = water_direct_act_numeric.mean(axis=1)
comparison_water_direct_df = pd.DataFrame({
    'Дата': daily_water_direct_pred.index,
    'Вода прямая (прогноз)': daily_water_direct_pred.values,
    'Вода прямая (факт)': daily_water_direct_act.values
})
comparison_water_direct_df['Дата'] = pd.to_datetime(comparison_water_direct_df['Дата'])

fig_line_water_direct = px.line(
    comparison_water_direct_df,
    x='Дата',
    y=['Вода прямая (прогноз)', 'Вода прямая (факт)'],
    title="Сравнение дневной средней температуры воды (прямая): прогноз vs факт",
    labels={'value': 'Температура (°C)', 'variable': 'Тип данных'}
)
fig_line_water_direct.write_html("graphs/line_comparison_water_direct.html")

In [ ]:
# Scatter plot для прямой линии
stacked_water_direct_pred = water_direct_pred_numeric.stack().reset_index(name='Вода прямая (прогноз)')
stacked_water_direct_act  = water_direct_act_numeric.stack().reset_index(name='Вода прямая (факт)')
merged_water_direct = pd.merge(stacked_water_direct_pred, stacked_water_direct_act, on=['level_0', 'level_1'])
merged_water_direct = merged_water_direct.rename(columns={'level_0': 'Дата', 'level_1': 'Комбинация'})
merged_water_direct['Дата'] = pd.to_datetime(merged_water_direct['Дата'])

fig_scatter_water_direct = px.scatter(
    merged_water_direct,
    x='Вода прямая (прогноз)',
    y='Вода прямая (факт)',
    color='Комбинация',
    hover_data=['Дата', 'Комбинация'],
    title="Scatter plot: Вода прямая – прогноз vs факт",
    labels={'Вода прямая (прогноз)': 'Прогноз', 'Вода прямая (факт)': 'Факт'}
)
min_val_direct = min(merged_water_direct['Вода прямая (прогноз)'].min(), merged_water_direct['Вода прямая (факт)'].min())
max_val_direct = max(merged_water_direct['Вода прямая (прогноз)'].max(), merged_water_direct['Вода прямая (факт)'].max())
fig_scatter_water_direct.add_trace(
    go.Scatter(
        x=[min_val_direct, max_val_direct],
        y=[min_val_direct, max_val_direct],
        mode='lines',
        line=dict(dash='dash', color='red'),
        name='y=x'
    )
)
fig_scatter_water_direct.write_html("graphs/scatter_comparison_water_direct.html")

## --- Графики для обратной линии воды (прогноз vs факт) ---
## Линейный график

In [ ]:
daily_water_reverse_pred = water_reverse_pred_numeric.mean(axis=1)
daily_water_reverse_act  = water_reverse_act_numeric.mean(axis=1)
comparison_water_reverse_df = pd.DataFrame({
    'Дата': daily_water_reverse_pred.index,
    'Вода обратная (прогноз)': daily_water_reverse_pred.values,
    'Вода обратная (факт)': daily_water_reverse_act.values
})
comparison_water_reverse_df['Дата'] = pd.to_datetime(comparison_water_reverse_df['Дата'])

fig_line_water_reverse = px.line(
    comparison_water_reverse_df,
    x='Дата',
    y=['Вода обратная (прогноз)', 'Вода обратная (факт)'],
    title="Сравнение дневной средней температуры воды (обратная): прогноз vs факт",
    labels={'value': 'Температура (°C)', 'variable': 'Тип данных'}
)
fig_line_water_reverse.write_html("graphs/line_comparison_water_reverse.html")

In [ ]:
# Scatter plot для обратной линии
stacked_water_reverse_pred = water_reverse_pred_numeric.stack().reset_index(name='Вода обратная (прогноз)')
stacked_water_reverse_act  = water_reverse_act_numeric.stack().reset_index(name='Вода обратная (факт)')
merged_water_reverse = pd.merge(stacked_water_reverse_pred, stacked_water_reverse_act, on=['level_0', 'level_1'])
merged_water_reverse = merged_water_reverse.rename(columns={'level_0': 'Дата', 'level_1': 'Комбинация'})
merged_water_reverse['Дата'] = pd.to_datetime(merged_water_reverse['Дата'])

fig_scatter_water_reverse = px.scatter(
    merged_water_reverse,
    x='Вода обратная (прогноз)',
    y='Вода обратная (факт)',
    color='Комбинация',
    hover_data=['Дата', 'Комбинация'],
    title="Scatter plot: Вода обратная – прогноз vs факт",
    labels={'Вода обратная (прогноз)': 'Прогноз', 'Вода обратная (факт)': 'Факт'}
)
min_val_reverse = min(merged_water_reverse['Вода обратная (прогноз)'].min(), merged_water_reverse['Вода обратная (факт)'].min())
max_val_reverse = max(merged_water_reverse['Вода обратная (прогноз)'].max(), merged_water_reverse['Вода обратная (факт)'].max())
fig_scatter_water_reverse.add_trace(
    go.Scatter(
        x=[min_val_reverse, max_val_reverse],
        y=[min_val_reverse, max_val_reverse],
        mode='lines',
        line=dict(dash='dash', color='red'),
        name='y=x'
    )
)
fig_scatter_water_reverse.write_html("graphs/scatter_comparison_water_reverse.html")